<a href="https://colab.research.google.com/github/Fuenfgeld/ATeamDatenmanagementUndArchivierung/blob/main/IMECOS_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# initialize the database, see import statements in *implementation.py
import requests
exec(requests.get('https://raw.githubusercontent.com/Fuenfgeld/ATeamDatenmanagementUndArchivierung/main/source_database_implementation.py').text)

Mounted at /content/gdrive
Successfully Connected to SQLite
Successfully created tables in the database
Successfully inserted source data
Source database ready for work
  
      conn    -    is the name of the connection object
      cur     -    is the name of the cursor
      
      


In [3]:
# Testabfrage
cur.execute("""SELECT *
               FROM tb_payer_transitions""")
testabfrage = cur.fetchall()
for row in testabfrage:
  print(row)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
('832abbf9-6cad-32ad-2211-fc964cc0f331', 'colorectal_cancer', 1967, 1985, 'd47b3510-2895-3b70-9897-342d681c769d', 'colorectal_cancer', 'Guardian', 'colorectal_cancer')
('4eefc17f-77b4-7e89-030e-a886b37b2ad8', 'colorectal_cancer', 1970, 1988, 'd47b3510-2895-3b70-9897-342d681c769d', 'colorectal_cancer', 'Guardian', 'colorectal_cancer')
('8fbfc792-1516-6b91-5d44-164f11e3f9bd', 'colorectal_cancer', 1940, 1958, '42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a', 'colorectal_cancer', 'Guardian', 'colorectal_cancer')
('8fbfc792-1516-6b91-5d44-164f11e3f9bd', 'colorectal_cancer', 1959, 2005, '7caa7254-5050-3b5e-9eae-bd5ea30e809c', 'colorectal_cancer', 'Self', 'colorectal_cancer')
('14a5d3cb-ab00-9258-54ce-bef2f8e50d96', 'colorectal_cancer', 2014, 2014, '7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a', 'colorectal_cancer', 'Self', 'colorectal_cancer')
('c7e4eacf-c074-3801-d62e-17563e496c34', 'colorectal_cancer', 1978, 1978, '7c4411ce-02f1-39b5-b9ec-df

In [5]:
#zeige alle tabellen
master_table_type = "SELECT name FROM sqlite_master WHERE type='table';" 
cur.execute(master_table_type)
print(cur.fetchall())


[('tb_patients',), ('tb_organizations',), ('tb_providers',), ('tb_payers',), ('tb_payer_transitions',), ('tb_encounters',), ('tb_careplans',), ('tb_conditions',), ('tb_devices',), ('tb_disease',), ('tb_imaging_studies',), ('tb_immunizations',), ('tb_observations',), ('sqlite_sequence',), ('tb_procedures',), ('tb_supplies',), ('tb_medications',)]


In [77]:
#SQL_Abfrage_patient_Id_unique
sql = "SELECT Id, count(Id) FROM tb_patients GROUP BY Id "
cur.execute(sql)
patient_Id_unique = cur.fetchall()
for row in patient_Id_unique:
  print(row)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
('4f1f1971-1c01-e5d3-f426-385bfe1672c9', 1)
('4f2485f3-7c69-f16c-7aad-a5133ad654b5', 1)
('4f3e066b-c984-08df-5d9c-4b029af9c1d6', 1)
('4f490250-2932-3ac5-a4e9-772c29100b2f', 1)
('4f52c315-2c16-a087-6aa1-87fca2160e4e', 1)
('4f5f887a-6662-3987-f463-2720fb5981e8', 1)
('4f61f946-e6c4-4d18-cb80-0704562bbe15', 1)
('4f75a040-83e4-8193-e491-b635c2cd135d', 1)
('4f87d75f-dac5-4278-36af-0f11232e7ec1', 1)
('4f8dc3e3-7d9a-bce1-567b-1101631858b1', 1)
('4f98f0ca-3786-0a1f-c028-cefc16791e72', 1)
('4f9feef7-1294-2233-87fb-216e837eaed7', 1)
('4fa452a2-20ea-6366-71ab-0b139fa3bf14', 1)
('4fad9995-ba7f-efa1-4a86-1edda40c069f', 1)
('4fae75ce-59f8-00d2-b53c-6ceb2299415c', 1)
('4fc0a5ea-210c-f051-448d-289adc0582c3', 1)
('4fd03445-a307-7cb6-3220-e78811da844f', 1)
('4fe5f187-e3ba-48ea-9935-266233fea6af', 1)
('4fe796ea-9b5e-f1d4-10c2-940ce40e6232', 1)
('4ff7a8f3-28c5-1e3a-601a-d96a498f0400', 1)
('4ffb04be-2621-4f50-c012-3b3fd3b90eef', 1)
('4ffeb12